In [26]:
%load_ext autoreload
%autoreload 2

In [10]:
import os
import jsonlines
import json
import pickle
from PIL import Image


path = "/mnt/disks/storage/data/finetune_data/9784kL.jsonl"
pickle_path = os.path.expanduser("~/image_paths.pkl")


image_paths = set()
if os.path.exists(pickle_path):
    with open(pickle_path, "rb") as f:
        image_paths = pickle.load(f)
        print(f"Loaded {len(image_paths)} image paths")

else:
    with jsonlines.open(path) as reader:
        for line in reader:
            obj = json.loads(line)
            if "image" in obj:
                image_paths.add(obj["image"])

# save as a pickle 
with open(pickle_path, "wb") as f:
    pickle.dump(image_paths, f)
    print(f"Saved {len(image_paths)} image paths")

print(f"Total number of image paths: {len(image_paths)}")


Loaded 3898574 image paths
Saved 3898574 image paths
Total number of image paths: 3898574


In [11]:
import os
import pickle
import json
from PIL import Image

# Parameters
HASH_SIZE = 8  # Should match the hash size used previously
THRESHOLD = 1  # Hamming distance threshold for matching
INDEX_FILENAME = "faiss_index_train_phash.index"  # Path to the FAISS index file
IMAGE_PATHS_PICKLE = os.path.expanduser("~/image_paths.pkl")  # Path to the training image paths
TEST_HASH_FILES_DIR = os.path.expanduser("~/test_hash_files_v2")  # Directory where test hashes are saved
RESULTS_DIR = os.path.expanduser("~/match_results_exact_v2")  # Directory to save the results

base_path = "/mnt/disks/storage/data/finetune_data/"


In [3]:
import json
path = "/mnt/disks/storage/data/finetune_data/jsons/updated_llava_instruct_150k_llavar_20k.json"
with open(path, "r") as f:
    data = json.load(f)

data

[]

In [1]:
from datasets import load_dataset, concatenate_datasets
from huggingface_hub import hf_hub_download

from dataset_hashes_v2 import test_dataset_map, get_dataset, preprocess_image, compute_hash, process_image, process_dataset, load_image

In [28]:


# Load match results for a specific dataset
# dataset_name = 'vstar'
dataset_name = 'mmbench_en'
results_file = os.path.join(RESULTS_DIR, f"{dataset_name}_matches.json")
with open(results_file, 'r') as f:
    data = json.load(f)

# Access match information
total_test_images = data['total_test_images']
n_test_images_with_match = data['n_test_images_with_match']
test_matches = data['test_matches']

print(f"{dataset_name}: {n_test_images_with_match}/{total_test_images} test images have matches in the training set.")

# For example, print matches for the first test image with matches
for test_idx, match_info in test_matches.items():
    print(f"Test image index: {test_idx}")
    print("Matched training indices:", match_info['matched_training_indices'])
    print("Matched training filenames:", match_info['matched_training_filenames'])
    break  # Remove this line to iterate over all matches


mmbench_en: 1122/4377 test images have matches in the training set.
Test image index: 3
Matched training indices: [375432]
Matched training filenames: ['/mnt/disks/storage/data/finetune_data/scienceqa/images/1682.jpg']


In [14]:
# Image.open(base_path + match_info['matched_training_filenames'][0])

In [27]:
# test_matches

In [29]:
# seed = load_dataset("imagefolder", data_dir="./resized_seed_bench_images", split="train")
# seed

KeyboardInterrupt: 

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt


def visualize_match(test_image, test_idx, training_image_paths,):
    fig, axes = plt.subplots(1, len(training_image_paths) + 1, figsize=(15, 5))
    axes[0].imshow(test_image)
    # axes[0].set_title('Test Image')
    axes[0].set_title(f"Test Image ({dataset_name} #{test_idx})")
    axes[0].axis('off')
    
    for idx, train_image_path in enumerate(training_image_paths):
        train_image_full_path = os.path.join(base_path, train_image_path)  # Adjust base_path as needed
        train_image = Image.open(train_image_full_path).convert('RGB')
        print(f"Path: {train_image_full_path}, type: {type(train_image)}")
        axes[idx + 1].imshow(train_image)
        axes[idx + 1].set_title(f'Training Image ({train_image_path})')
        axes[idx + 1].axis('off')
    
    plt.show()

# Example usage

dataset = get_dataset(dataset_name)
for test_idx, match_info in test_matches.items():
    test_row = dataset.select([int(test_idx)])
    test_key = test_dataset_map[dataset_name][3]
    test_image = load_image(test_row, test_key)
    training_image_paths = match_info['matched_training_filenames']
    visualize_match(test_image, test_idx, training_image_paths)


In [21]:
test_matches

{'332': {'matched_training_indices': [325980],
  'matched_training_filenames': ['synthdog/images/186479.jpg']}}

In [5]:
train_hashes

[10329161939788197919,
 11022592825073351645,
 13563790271933347896,
 14068186166199818553,
 11184979863083155694,
 10653610698513976742,
 16261662366178108638,
 9240679752006729411,
 16242735849712563356,
 16402563127497909565,
 9828311359185729131,
 16565513160477283116,
 16321219112698176823,
 18025870939612152759,
 12597641472525154100,
 12155263818385555431,
 11671122933066746266,
 9732031895881432126,
 12362445373625315706,
 18349218950347782789,
 10055326275180327368,
 9264870309846219150,
 9231850599180169184,
 12953638198352032591,
 10020364577654546814,
 9329909031353087869,
 11931822275950254875,
 13408105493956017565,
 13984026211912020579,
 17422775592678293897,
 17560541417700532601,
 15755197173407982922,
 18197509619189462935,
 12406555449197056915,
 16991106719187751286,
 15721453811815477280,
 15047471609651605682,
 16121083630613463870,
 12589448075654287657,
 10960586517958448064,
 13422715111805529022,
 9332582508844581756,
 9906938035579378953,
 136508522391110489

In [2]:
import os
import pickle
from PIL import Image
import imagehash

# Parameters
HASH_SIZE = 8  # Should match the hash size used previously
TRAIN_HASH_FILES_DIR = os.path.expanduser("~/hash_files")  # Directory where train hashes are saved
IMAGE_PATHS_PICKLE = os.path.expanduser("~/image_paths.pkl")  # Path to the training image paths
BASE_IMAGE_PATH = "/mnt/disks/storage/data/finetune_data/"  # Base path to your images

# Load all training image paths
with open(IMAGE_PATHS_PICKLE, "rb") as f:
    image_paths = pickle.load(f)
image_paths = list(image_paths)  # Ensure it's a list
total_train_images = len(image_paths)
print(f"Loaded {total_train_images} training image paths")

# Select a train hash file to load
# For example, let's pick the first hash file
hash_file_name = "hashes_0.pkl"  # Replace with the actual filename
hash_file_path = os.path.join(TRAIN_HASH_FILES_DIR, hash_file_name)

# Load the hashes from the selected hash file
with open(hash_file_path, 'rb') as f:
    train_hashes = pickle.load(f)
num_hashes_in_file = len(train_hashes)
print(f"Loaded {num_hashes_in_file} hashes from {hash_file_name}")

# Select an index within this hash file to test
# For example, let's pick the first hash
hash_index_in_file = 0  # Change this index to test other images

# Compute the global index of the image in image_paths
# This depends on how you assigned indices during hashing
# If you processed batches sequentially, the global index is:
# global_index = (batch_number * batch_size) + hash_index_in_file

# Extract the batch number from the hash file name
# Assuming the hash file names are like 'hashes_<batch_number>.pkl'
batch_number_str = hash_file_name.replace('hashes_', '').replace('.pkl', '')
batch_number = int(batch_number_str)

# Parameters used during processing
BATCH_SIZE = 10000  # Should match the batch size used during processing

# Compute the global index
global_index = (batch_number * BATCH_SIZE) + hash_index_in_file

# Ensure the global index is within bounds
if global_index >= total_train_images:
    print(f"Global index {global_index} is out of bounds.")
else:
    # Get the image path from image_paths
    image_rel_path = image_paths[global_index]
    image_path = os.path.join(BASE_IMAGE_PATH, image_rel_path)
    print(f"Image path: {image_path}")

    # Load and preprocess the image
    try:
        with Image.open(image_path) as image:
            # Preprocess the image
            image = image.convert('L')            # Convert to grayscale
            image = image.resize((256, 256))      # Resize to a standard size

            # Compute the hash
            hash_obj = imagehash.phash(image, hash_size=HASH_SIZE)
            computed_hash_int = int(str(hash_obj), 16)

            # Get the hash from the hash file
            stored_hash_int = int(train_hashes[hash_index_in_file])

            # Compare the hashes
            print(f"Computed hash: {computed_hash_int}")
            print(f"Stored hash:   {stored_hash_int}")

            if computed_hash_int == stored_hash_int:
                print("Hashes match.")
            else:
                print("Hashes do not match.")

    except Exception as e:
        print(f"Error loading or processing image: {e}")


Loaded 3898574 training image paths
Loaded 10000 hashes from hashes_0.pkl
Image path: /mnt/disks/storage/data/finetune_data/allava/ALLaVA-4V/allava_laion/image_chunks/images/46675190.jpeg
Computed hash: 13675978674883051926
Stored hash:   10329161939788197919
Hashes do not match.


In [4]:
image_paths

['allava/ALLaVA-4V/allava_laion/image_chunks/images/46675190.jpeg',
 'synthdog/images/496233.jpg',
 'synthdog/images/482954.jpg',
 'websight/images/train_13766.png',
 'allava/ALLaVA-4V/allava_laion/image_chunks/images/79741.jpeg',
 'synthdog/images/8408.jpg',
 'allava/ALLaVA-4V/allava_laion/image_chunks/images/46228731.jpeg',
 'llavar/finetune/100000563893.jpg',
 'dvqa/images/bar_train_00130199.png',
 'dvqa/images/bar_train_00105926.png',
 'websight/images/train_178788.png',
 'websight/images/train_347477.png',
 'lnqa/images/train_160661.jpg',
 'synthdog/images/241615.jpg',
 'vg/VG_100K_2/2417404.jpg',
 'websight/images/train_240654.png',
 'lnqa/images/train_181120.jpg',
 'textvqa/train_images/7aa63ec52620bdd5.jpg',
 'websight/images/train_685127.png',
 'lnqa/images/train_52889.jpg',
 'hfdata/tabmwp/1615.jpg',
 'arxivqa/images/2207.13678_1.jpg',
 'allava/ALLaVA-4V/allava_laion/image_chunks/images/100801661.jpeg',
 'allava/ALLaVA-4V/allava_laion/image_chunks/images/136036306.jpeg',
 'we